## Write Data to HDFS using Spark

Let us now write the data to HDFS using Spark Structured Streaming APIs. It will take care of writing streaming data frame to HDFS.

Here are the steps that are involved.
* Create spark session object.
* Create streaming data frame by subscribing to Kafka topic leveraging `readStream`.
* Apply required transformations to add new fields such as year, month and dayofmonth.
* Write the data from streaming data frame to HDFS using appropriate format. We will be using `csv` for now. We can write data to all standard formats using streaming data frame.
* We need to specify `checkpointLocation` while writing data to HDFS. The location should be in HDFS itself.
* As part of this lecture we will validate whether the files are being generated or not. We will perform detailed validation as part of next lecture.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1'). \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Kafka and Spark Integration'). \
    master('yarn'). \
    getOrCreate()

In [2]:
kafka_bootstrap_servers = 'w01.itversity.com:9092,w02.itversity.com:9092'

In [3]:
df = spark. \
  readStream. \
  format('kafka'). \
  option('kafka.bootstrap.servers', kafka_bootstrap_servers). \
  option('subscribe', f'{username}_retail'). \
  load()

In [4]:
from pyspark.sql.functions import date_format, to_date, split, substring

In [6]:
df.selectExpr("CAST(key AS STRING) AS key", "CAST(value AS STRING) AS value").printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [7]:
!hdfs dfs -rm -R -skipTrash /user/${USER}/kafka/retail_logs/

rm: `/user/itv007304/kafka/retail_logs/': No such file or directory


In [7]:
df.selectExpr("CAST(value AS STRING)"). \
    withColumn('log_date', to_date(substring(split('value', ' ')[3], 2, 21), '[dd/MMM/yyyy:HH:mm:ss')). \
    withColumn('year', date_format('log_date', 'yyyy')). \
    withColumn('month', date_format('log_date', 'MM')). \
    withColumn('dayofmonth', date_format('log_date', 'dd')). \
    writeStream. \
    partitionBy('year', 'month', 'dayofmonth'). \
    format('csv'). \
    option("checkpointLocation", f'/user/{username}/kafka/retail_logs/gen_logs/checkpoint'). \
    option('path', f'/user/{username}/kafka/retail_logs/gen_logs/data'). \
    trigger(processingTime='30 seconds'). \
    start()

In [8]:
!hdfs dfs -ls /user/${USER}/kafka/retail_logs/gen_logs

Found 2 items
drwxr-xr-x   - itversity itversity          0 2021-09-02 13:20 /user/itversity/kafka/retail_logs/gen_logs/checkpoint
drwxr-xr-x   - itversity itversity          0 2021-09-02 13:20 /user/itversity/kafka/retail_logs/gen_logs/data


In [9]:
!hdfs dfs -ls -R /user/${USER}/kafka/retail_logs/gen_logs/data

drwxr-xr-x   - itversity itversity          0 2021-09-02 13:20 /user/itversity/kafka/retail_logs/gen_logs/data/_spark_metadata


In [8]:
df.selectExpr("CAST(value AS STRING)"). \
    withColumn('log_date', to_date(substring(split('value', ' ')[3], 2, 21), '[dd/MMM/yyyy:HH:mm:ss')). \
    withColumn('year', date_format('log_date', 'yyyy')). \
    withColumn('month', date_format('log_date', 'MM')). \
    withColumn('dayofmonth', date_format('log_date', 'dd')). \
    writeStream. \
    partitionBy('year', 'month', 'dayofmonth'). \
    format('csv'). \
    option("checkpointLocation", f'/user/{username}/kafka/retail_logs/gen_logs/checkpoint'). \
    option('path', f'/user/{username}/kafka/retail_logs/gen_logs/data'). \
    option('header', True). \
    option('sep', '\t'). \
    trigger(processingTime='30 seconds'). \
    start()

In [9]:
!hdfs dfs -ls /user/${USER}/kafka/retail_logs/gen_logs

Found 2 items
drwxr-xr-x   - itv007304 supergroup          0 2023-06-09 14:17 /user/itv007304/kafka/retail_logs/gen_logs/checkpoint
drwxr-xr-x   - itv007304 supergroup          0 2023-06-09 14:17 /user/itv007304/kafka/retail_logs/gen_logs/data


In [13]:
!hdfs dfs -ls -R /user/${USER}/kafka/retail_logs/gen_logs/data

drwxr-xr-x   - itv007304 supergroup          0 2023-06-09 14:26 /user/itv007304/kafka/retail_logs/gen_logs/data/_spark_metadata
-rw-r--r--   3 itv007304 supergroup          2 2023-06-09 14:17 /user/itv007304/kafka/retail_logs/gen_logs/data/_spark_metadata/0
-rw-r--r--   3 itv007304 supergroup        893 2023-06-09 14:25 /user/itv007304/kafka/retail_logs/gen_logs/data/_spark_metadata/1
-rw-r--r--   3 itv007304 supergroup        893 2023-06-09 14:26 /user/itv007304/kafka/retail_logs/gen_logs/data/_spark_metadata/2
-rw-r--r--   3 itv007304 supergroup        893 2023-06-09 14:26 /user/itv007304/kafka/retail_logs/gen_logs/data/_spark_metadata/3
drwxr-xr-x   - itv007304 supergroup          0 2023-06-09 14:25 /user/itv007304/kafka/retail_logs/gen_logs/data/year=2023
drwxr-xr-x   - itv007304 supergroup          0 2023-06-09 14:25 /user/itv007304/kafka/retail_logs/gen_logs/data/year=2023/month=02
drwxr-xr-x   - itv007304 supergroup          0 2023-06-09 14:26 /user/itv007304/kafka/retail_logs/g

In [ ]:
!hdfs dfs -cat /user/itv007304/kafka/retail_logs/gen_logs/data/year=2023/month=02/dayofmonth=23/part-00002-ff8e3398-38c8-46fe-ac66-8edf68c9c05a.c000.csv

In [18]:
df = spark.read.csv(f"/user/{username}/kafka/retail_logs/gen_logs/data", sep='\t', header=True)

In [19]:
df.printSchema()

root
 |-- value: string (nullable = true)
 |-- log_date: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)



In [20]:
df.show(10, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----+-----+----------+
|value                                                                                                                                                                                                                   |log_date  |year|month|dayofmonth|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----+-----+----------+
|109.150.11.154 - - [23/Feb/2023:12:17:38 -0800] "GET /login HTTP/1.1" 200 283 "-" "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"                       |2023-02-23|2023|2    |23  